In [ ]:
import rasterio
import glob
from matplotlib import pyplot
import json
from IPython.display import clear_output
from matplotlib import pyplot


window_size = 256
offset = (window_size/2)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def get_window(x, y):
    return data[int(y-offset):int(y+offset), int(x-offset):int(x+offset)]

def show_sample(sample):
    d = get_window(sample['pos'][1]*8, sample['pos'][0]*8)
    pyplot.imshow(d, cmap='viridis', vmin=0, vmax=256)
    pyplot.plot(int(window_size/2), int(window_size/2), marker="x", markersize=3, markeredgecolor="red", markerfacecolor="red")
    pyplot.show()

with open('samples.json', 'r') as f:
    samples = json.load(f)
    
    for period, period_samples in samples.items():
        with rasterio.open(f'output/{period}_ndwi_clipped.tif', 'r') as src:
            data = src.read(1)

        for i, sample in enumerate(period_samples):
            clear_output(wait=True)
            if sample['water'] is not None:
                continue
            print(f'{i+1}/{len(period_samples)}')
            pyplot.figure(figsize=(8, 8))
            show_sample(sample)

            isWater = input('Water? (y/n)') == 'y'
            period_samples[i]['water'] = isWater

            with open('samples.json', 'w') as f:
                f.write(json.dumps(samples, indent=4, cls=NpEncoder))